<a href="https://colab.research.google.com/github/hursoo/big_k-modern_1/blob/main/gb_021_preprocess(big1_251)_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.필요한 도구와 라이브러리 설치

In [ ]:
# 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 경로 지정

file_path = '/content/drive/MyDrive/_big1_25-1_code/'

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import re

# 2.데이터 불러오기

In [ ]:
# 데이터 불러오기

df = pd.read_excel(file_path + 'result/ron10_data.xlsx')
print(df.shape)

df

In [ ]:
df1 = df[['r_id', 'title', 'content']]
df1

# 3.기사를 문장별로 분리

In [ ]:
import re

# raw string 사용
separators = [r'\n', r'\.', r'!', r'\?']
pattern = '|'.join(separators)  # 바로 윗 줄 구분자들을 '또는'으로 연결함

df1 = df1.copy()
df1.loc[:, 'splited_sent'] = df1['content'].apply(lambda x: re.split(pattern, x))
df1

In [ ]:
# 각 문장을 df의 행이 되도록 변환

sents_df = df1.explode('splited_sent')
sents_df = sents_df.reset_index(drop=True) # 인덱스를 재부여
sents_df

# 4.형태소 분석
- 제목: 형태소 분석기 적용
- kiwi 형태소 분석기로 텍스트 데이터 분석

## 4.1.kiwi 설치

In [ ]:
# kiwi 설치
!pip install -U kiwipiepy

## 4.2.형태소 분석 함수 작성

In [ ]:
from kiwipiepy import Kiwi

# Kiwi 객체 생성
kiwi = Kiwi()

# 텍스트 분석
text = "소리-있어 넓히 世界에 傳하니 온 世界 모든 人類-이에 應하야 부르짖기를 始作하도다"
results = kiwi.analyze(text)

results

In [ ]:
# 영어, 숫자, 특정 부호 등 제외하기
def filter_elements(lst):
    # 영어, 숫자 및 '<' 문자만을 허용하는 정규식 패턴
    pattern = re.compile(r'^[a-zA-Z0-9<＞「」]+$')

    # 해당 패턴과 일치하지 않는 문자열만을 반환하는 필터링 함수
    return list(filter(lambda x: not pattern.match(x), lst))

# 형태소 분석 결과를 단순한 형태로 출력하는 사용자 함수
def simplify_kiwi(sent):
    morphed_list = []
    results = kiwi.analyze(sent)
    for result in results:
        for token in result[0]:  # result는 (token_list, int) 형태의 튜플입니다.
            morphed = (token.form + "/" + token.tag)  # 각 토큰의 형태소와 품사 태그 출력
            morphed_list.append(morphed)
    return morphed_list

# 본격 형태소 분석
import string
def do_morph(sent):
    # 토큰나이징 결과 확인 (subword + 한국어 형태소)
    vocab_list = simplify_kiwi(sent)
    result01 = filter_elements(vocab_list) # 영어, 숫자, 특정부호 제외 (사용자 함수)
    return result01

## 4.3.형태소 분석 및 정리 정돈

In [ ]:
print(sents_df.shape)
sents_df.head(5)

In [ ]:
# 'content' 열에 kiwi 형태소 분석기 적용
sents_df = sents_df.copy()
sents_df['tokens'] = sents_df['splited_sent'].apply(do_morph)
sents_df

In [ ]:
# 'tokens' 열에서 리스트 요소가 하나도 없는 행 제거
sents_df1 = sents_df[sents_df['tokens'].str.len() > 0]

sents_df1 = sents_df1.copy()
sents_df1['tokens'] = sents_df1['tokens'].apply(lambda x: ' '.join(x)) # 리스트를 문자열로
sents_df1

In [ ]:
sents_df1['morph'] = sents_df1['tokens'].apply(
    lambda x: [token.split('/')[0] for token in x.split() if token.split('/')[-1] in ['SH', 'NP', 'NNP']]
)

# 'morph' 열에서 리스트 요소가 하나도 없는 행 제거
sents_df2 = sents_df1[sents_df1['morph'].str.len() > 0]

# 리스트를 문자열로
sents_df2 = sents_df2.copy()
sents_df2['morph'] = sents_df2['morph'].apply(lambda x: ' '.join(x))

print(sents_df2.shape)
sents_df2.head(5)

In [ ]:
# 문장 인덱스 생성
sents_df2 = sents_df2.reset_index(drop = True)
sents_df2['sent_id'] = sents_df2.index + 1

# 열 순서 변경
sents = sents_df2[['sent_id', 'morph', 'splited_sent', 'r_id', 'title', 'content']]
sents

# The End of Notebook